Guía Uso modelo SHIA por medio del paquete WMF. Escala horaria o menor

# Insumos

## Paquetes a importar

In [1]:
%matplotlib inline 
import pylab as pl 
from wmf import wmf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as pl
import warnings
import os
import datetime
import netCDF4
import glob
warnings.filterwarnings('ignore')

## Datos necesarios

* Modelo de Elevación digital
* Mapa de Direcciones (en formato qgis)
* Coordenadas x,y del punto de trazado de cuenca
* Mapa de conductividad hidráulica Ks (mm/h)
* Mapa de almacenamiento máximo capilar Hu
* Mapa de almacenamiento máximo gravitacional Hg
* Mapa de coeficiente de Manning
* Serie de caudales en el punto de trazado de la cuenca y en la resolución temporal de la simulación (dt)
* Información de lluvia dentro de la cuenca en la misma resolución temporal de la simulación

# Procesamiento de información de entrada al modelo

## Cuenca, DEM y mapa de direcciones

Partiendo del DEM, del mapa de direcciones y del punto de trazado se ejecutan los siguientes comandos:

DEM=wmf.read_map_raster(rutaDEM, isDEMorDIR=True, dxp=dxp, noDataP=noDataP)[0]
DIR=wmf.read_map_raster(rutaDIR, isDEMorDIR=True, dxp=dxp, noDataP=noDataP,isDIR = True)[0]

st = wmf.Stream(x, y, DEM, DIR)
cu = wmf.SimuBasin(x, y, DEM, DIR, stream = st, umbral = umbral_red, noData=noDataP, dt=deltaT)

* El umbral se define como el número de pixeles acumulados que drenan a una celda para que se considere dentro de la red de drenaje
* dt en segundos y dxp en metros


En caso de ya tener un nc de la cuenca se llama así:

In [2]:
cu = wmf.SimuBasin(rute = '/media/nicolas/Home/Jupyter/sedimentos/porce/datos/cuencas/porce_150m.nc')

## Parámetros de entrada requeridos para la simulación del transporte vertical

### Cálculo de la evaportranspiración (Salida T1):

* Esta formulación es de Cenicafé para las cuencas de los ríos Cauca y Magdalena en la zona Andina de Colombia.
* Se divide por 86400 para realizar la conversión a [mm/s]

In [4]:
cu.GetGeo_Cell_Basics()

Evp=4.658*np.exp(-0.0002*cu.CellHeight)/86400.0# en mm/s

### Conductividad hidráulica del suelo residual Ks (Entrada T3 como infiltración):

* Ks en [mm/h], luego se realiza la conversión

In [23]:
Ks, p,epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/Alexa/flood_forecast/data/Folder_parametros/Ks.tif')
Ks = cu.Transform_Map2Basin(Ks,p)

Ks[Ks == -9999] = Ks[Ks>0].mean()
Ks[Ks == 0] = Ks[Ks>Ks[Ks!=0].min()].min()
Ks[Ks>32.4]=32.4
Ks[Ks<=0.0]=Ks[Ks>0].min()

### Conductividad hidráulica del saprolito Kp (Entrada T4 como percolación):

Se asume igual a:

In [24]:
Kp = np.copy(Ks/100.0)

## Almacenamientos máximos

### Máximo almacenamiento capilar (almacenamiento máximo e T1):

* Hu en [mm]

In [25]:
Zcap,p,epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/Alexa/flood_forecast/data/Folder_parametros/Hu.tif')
Zcap = cu.Transform_Map2Basin(Zcap,p)
Zcap[Zcap<0]=np.min(Zcap[Zcap>0])

### Máximo almacenamiento gravitacional (almacenamiento máximo en T3):


* Hg en [mm]

In [26]:
Zg,p,epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/Alexa/flood_forecast/data/Folder_parametros/Hg.tif')
Zg = cu.Transform_Map2Basin(Zg,p)
Zg[Zg<=0]=np.min(Zg[Zg>0])

## Coeficientes de entrada requeridos para la simulación del transporte horizontal

### Coeficiente de Foster para el cálculo del fujo de escorrentía (T2)

* Recordar que  

$$
V_{2}=\frac{\zeta A^{\left(2 / 3 e_{1}\right)} S^{0.5}}{n} \text {  y el coeficiente de Foster será   } \frac{\zeta S^{0.5}}{n}
$$

* Para surcos (Foster y Lane, 1981) en (Vélez, 2001) proponen $\zeta$=0.5 y $e_1$=0.64

In [29]:
Man, p,epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/Alexa/flood_forecast/data/Folder_parametros/Manning.tif')
Man = cu.Transform_Map2Basin(Man,p)
Man[Man <= 0] = 0.01

CoefRun = (0.5/Man) * cu.CellSlope**0.5
CoefRun[CoefRun>15] = 15

### Coeficiente Ksh para el cálculo del flujo subsuperficial horizontal (T3)

* Recordar que 
$$v = C_o A^{\alpha}$$
$$
V_{3}=\frac{K_{S} S A(t)^{b}}{(b+1) A_{g}^{b}}, \text { el coeficiente $C_o$ que se ingresa es: } K_{s h}=\frac{K_{S} S d x^{2}}{(b+1) H_{g}^{b}}
$$

* De acuerdo a los autores  𝑏=2  para cuencas de montaña con conducciones internas heterogeneas.
* La unidades de la ecuación anterior deben llegar a ser de: $m/s$

Unidades:

- $ks$: $mm/hora$
- $So$: Adimensional.
- $\Delta x$: $m$.
- $H_g$: $mm$.
- $A(t)$: $mm$.

$A(t)$ sebe ser expresado en términos de volúmen, igualmente $H_g$, por lo que se usa el factor de conversión basado en el área de un elemento:

- Factor de conversión $F_c = \Delta x ^2 \frac{1m}{1000mm}[m^3/mm]$

- $ks$ pasa a unidades de $m/s$ al ser dividido por $3.6x10^6$.
- $Hg$ pasa a $m^3$ al ser multiplicado por $F_c$.
- $S_3$ pasa a $m^3$ al ser multiplicado por $F_c$.

Con lo anterior se tiene que las unidades del coeficiente $C_o$ son: $s^{-1}m^{-3}$.  Las unidades de $A^{\alpha}$ son $m^4$.  Con lo anterior se obtienen unidades de velocidad $m/s$. 

In [34]:
Fc = (wmf.cu.dxp**2)/1000.

CoefSub=((Ks/3.6e6)*cu.CellSlope*(wmf.cu.dxp**2.0))/(3*(Zg*Fc)**2)

### Coeficiente para el cálculo del flujo en cauces:

La velocidad en el cauce se deduce a partir de la onda cinematica geomorfológica (OCG) y relaciones de geometría hidráulica. Para esto, al modelo se le ingresan los mapas de áreas acumuladas y pendientes y este cácula los coeficientes así

In [36]:
area = cu.CellAcum * (wmf.cu.dxp**2)/1e6
CoefOCG,ExpOCG = wmf.OCG_param(pend = cu.CellSlope, area = area)

In [41]:
CoefByTramo = np.zeros(cu.ncells)
for i in range(1,cu.nhills):
    pos = np.where((cu.hills_own == i) & (cu.CellCauce == 1))
    CoefByTramo[pos] = np.median(CoefOCG[pos])

# Configuración del modelo con parámetros calculados en 2

## Transporte vertical:

Los parámetros de transporte vertical se nombran como [‘v_coef,’ parámetro, i], donde i indica el proceso al que se hace referencia así:
[0: Tasa de evaporación, 1:Infiltración, 2:Percolación, 3:Pérdidas].
	
Cada uno de ellos se debe ingresar en [mm/s]


In [37]:
cu.set_PhysicVariables('v_coef', Evp, 0)
cu.set_PhysicVariables('v_coef', Ks/3600, 1)
cu.set_PhysicVariables('v_coef', Kp/3600, 2)
cu.set_PhysicVariables('v_coef', 0, 3)

## Almacenamientos:

* Los almacenamientos capilar [0] y gravitacional [1] siempre se ingresan al modelo en [mm]
* Además, se agrega el almacenamiento máximo en acuifero

In [38]:
cu.set_PhysicVariables('capilar', Zcap, 0)
cu.set_PhysicVariables('gravit', Zg, 1)
wmf.models.max_aquifer=wmf.models.max_gravita*10

## Transporte horizontal

En primer lugar se especifica el tipo de velocidad a usar en cada nivel del modelo, así:

In [39]:
cu.set_Speed_type([2,2,1])

Cada numero indica un tanque diferente [superficial, subsuperficial,subterraneo]
* 1: velocidad lineal, en este caso no se debe indicar h_exp
* 2: velocidad onda cinemática, se debe indicar h_exp

Cuando se usa la opción por OCG las velocidades son calculadas así:
$$v = h_{coef}(A^{h_{exp}})$$


Los parámetros del transporte horizontal se dividen en dos, coeficientes y exponentes y se nombran como h_coef y h_exp respectivamente. Además también cuentan con un número indicativo para cada proceso.
[0: Flujo de escorrentía, 1:Flujo subsuperficial, 2: Flujo subterráneo, 3: Flujo en cauces]


In [43]:
cu.set_PhysicVariables('h_coef',CoefRun, 0)
cu.set_PhysicVariables('h_exp', 0.64 , 0)

cu.set_PhysicVariables('h_coef',CoefSub, 1)
cu.set_PhysicVariables('h_exp',2.0, 1)

cu.set_PhysicVariables('h_coef', Kp/3600, 2)

cu.set_PhysicVariables('h_coef', CoefByTramo, 3, mask=cu.CellCauce)
cu.set_PhysicVariables('h_exp', ExpOCG, 3, mask=cu.CellCauce)

* El exponente en el flujo de escorrentía igual a 0.64 se propone por (Foster y Lane, 1981) para surcos, 

# Guardar archivo de configuración para modelo hidrológico unciamente

Finalmente se da la opción de guardar el archivo de configuración de la siguiente forma:

In [44]:
cu.Save_SimuBasin('/media/nicolas/Home/Jupyter/Stephany/ModeloHidrologico/Porce/Porce15min_150m_SoloHidrologico.nc')

# Sedimentos

Para incluir la simulación de sedimentos dentro del archivo de configuración es necesario tener los siguientes insumos:

* Mapa del factor K (erosividad del suelo) 
* Mapa del factor C (cobertura del suelo)
* Mapa del factor P (practicas de conservación del suelo)
* Mapa de porcentaje de arcillas, limos y arenas

In [45]:
## K de la RUSLE
k_rusle, prop, epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/sedimentos/porce/datos/raster/k_rusle_150m.tif', dxp = 150, noDataP = -9999)
k_rusle = cu.Transform_Map2Basin(k_rusle, prop)
k_rusle[k_rusle<0]=0

## C de la RUSLE
c_rusle, prop, epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/sedimentos/porce/datos/raster/c_rusle_150m.tif', dxp = 150, noDataP = -9999)
c_rusle = cu.Transform_Map2Basin(c_rusle, prop)

## Porcentaje de arenas limos y arcillas 
Ar, prop, epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/sedimentos/porce/datos/raster/arenas_150m.tif', dxp = 150, noDataP = -9999)
Ar = cu.Transform_Map2Basin(Ar, prop)
Ar[Ar<0]=0
Ac, prop, epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/sedimentos/porce/datos/raster/arcillas_150m.tif', dxp = 150, noDataP = -9999)
Ac = cu.Transform_Map2Basin(Ac, prop)
Ac[Ac<0]=0
Li, prop, epsg = wmf.read_map_raster('/media/nicolas/Home/Jupyter/sedimentos/porce/datos/raster/limos_150m.tif', dxp = 150, noDataP = -9999)
Li = cu.Transform_Map2Basin(Li, prop)
Li[Li<0]=0
Parliac = np.vstack([Ar, Li, Ac])*100.


## Ingreso Parámetros al modelo
cu.set_sediments(Parliac, 'PArLiAc')
cu.set_sediments(k_rusle, 'Krus')
cu.set_sediments(c_rusle, 'Crus')
cu.set_sediments(1., 'Prus')

In [48]:
cu.Save_SimuBasin('/media/nicolas/Home/Jupyter/Stephany/ModeloHidrologico/Porce/Porce15min_150m_Sedimentos.nc')